# SQL BAGU 

## **Why single-node databases can’t handle Big Data**

* **Storage limit:** Single-node systems have finite disk and memory.
* **Compute limit:** Single CPU/machine cannot efficiently process petabytes of data.
* **Fault tolerance:** Single-node failure causes complete downtime.

---

## **Problems solved by distributed systems & new challenges**

* **Solves:**

  * Horizontal scalability
  * Fault tolerance / high availability
  * Parallel processing for large datasets

* **Challenges:**

  * Network latency
  * Data consistency (CAP theorem)
  * Distributed transactions
  * Debugging and monitoring complexity

---

## **OLTP vs OLAP**

* **OLTP:**
  * Online Transaction Processing
  * Real-Time,Transactional, row-based, frequent small writes/reads, save original and standard data 
  * Example: Banking, e-commerce

* **OLAP:**
  * Online Analytical Processing 
  * Analytical, columnar, large aggregations through ETL based on OLTP
  * Example: Business intelligence, reporting

* **Design goal:** 
  * OLTP → fast transactional integrity; 
  * OLAP → fast analytical queries

OLTP： 
- 用途：处理日常事务型操作，强调 高并发、快速写入/更新/查询单条记录
- 特点：实时性强，表结构通常规范化
- 场景：银行转账、购物下单、库存更新

OLAP：
- 用途：用于分析和决策支持，强调 复杂查询、聚合、统计
- 特点：查询慢但复杂，表结构通常星型/雪花型，存历史数据
- 场景：销售数据分析、用户行为统计、财务报表

---

## **ETL vs ELT**

* **ETL:** Extract → Transform → Load
  * Transform before loading; good for traditional warehouses
* **ELT:** Extract → Load → Transform
  * Transform in data warehouse (e.g., Snowflake, Redshift); suits cloud-scale

* **Reason ELT is preferred in cloud:** Scalability, compute separation, easier schema changes

- ETL (Extract-Transform-Load)
流程：先 抽取(Extract) → 转换(Transform) → 加载(Load)
特点：数据在进入目标仓库前就已经清洗、转换好
适用场景：传统数据仓库，目标库计算能力有限
- ELT (Extract-Load-Transform)
流程：先 抽取(Extract) → 加载(Load) → 在目标库转换(Transform)
特点：依赖目标库强大的计算能力（如云仓库、分布式系统）进行转换
适用场景：现代数据湖/云仓库，大数据分析场景

| 特性    | ETL           | ELT           |
| ----- | ------------- | ------------- |
| 转换位置  | 在源或中间处理       | 在目标仓库处理       |
| 适合数据量 | 小到中等          | 大数据量          |
| 依赖    | 中间服务器或 ETL 工具 | 目标数据库/仓库计算能力强 |
| 数据延迟  | 高，转换耗时        | 低，可直接加载原始数据   |

ELT Code Sampe
```python
import pandas as pd
import sqlite3

# Extract & Load
data = pd.read_csv('orders.csv')  # 抽取
conn = sqlite3.connect('olap.db')
data.to_sql('orders_raw', conn, if_exists='replace', index=False)  # 直接加载

# Transform（在数据库里执行）
cur = conn.cursor()
cur.execute('''
UPDATE orders_raw
SET total_price = quantity * price
WHERE total_price IS NULL
''')
conn.commit()
```


---

## **MapReduce execution flow**

* **Map:** Process input splits, produce (key, value) pairs
* **Shuffle/Sort:** Group values by key, redistribute across nodes
* **Reduce:** Aggregate grouped values, write output

### 1. 概念总结

* **MapReduce** 是大数据分布式计算框架的核心编程模型
* **目标**：处理大规模数据，通过 **Map（映射）** 和 **Reduce（归约）** 两步完成
* **特点**：可扩展到成千上万节点，自动处理数据分片和容错

### 2. 执行流程分解

#### **1：Input Splitting（输入分片）**

* 将大文件切成多个 **Input Split**
* 每个 split 分配给一个 **Map Task**

#### **2：Mapping（映射）**

* Map 函数处理每个 split
* **输入**：`<key, value>` 对
* **输出**：中间 `<key, value>` 对

#### **3：Shuffling & Sorting（洗牌和排序）**

* 系统自动将 Map 输出根据 key 分组
* 相同 key 的数据会发送到同一个 Reduce Task

#### **4：Reducing（归约）**

* Reduce 函数对相同 key 的所有 values 进行处理
* 输出最终结果 `<key, aggregated_value>`

#### **5：Output（输出结果）**

* Reduce 输出写入 HDFS 或目标存储

### 3. 图示流程（文字版）

```
[Input File]
      │
      ▼
[Input Split] → 分配给多个 Map Task
      │
      ▼
[Map Function] → 生成中间 <key, value>
      │
      ▼
[Shuffle & Sort] → 相同 key 聚合
      │
      ▼
[Reduce Function] → 汇总处理
      │
      ▼
[Output File]
```

### 4. Python 模拟示例（Word Count）

```python
from collections import defaultdict

#   1 & 2: Map
data = ["hello world", "hello mapreduce"]
intermediate = []
for line in data:
    words = line.split()
    for word in words:
        intermediate.append((word, 1))  # Map 输出 <word, 1>

# Step 3: Shuffle & Sort
shuffle_sort = defaultdict(list)
for key, value in intermediate:
    shuffle_sort[key].append(value)

# Step 4: Reduce
result = {}
for key, values in shuffle_sort.items():
    result[key] = sum(values)  # Reduce 聚合

print(result)  # 输出: {'hello': 2, 'world': 1, 'mapreduce': 1}
```

> 核心：Map 负责拆和标记，Shuffle 按 key 聚合，Reduce 汇总计算。


---

###### **Data Lake vs Data Warehouse**

* **Data Lake:** Raw, unstructured/semi-structured data, low cost, flexible

* **Data Warehouse:** Structured, cleaned, optimized for queries, high performance

* **Use case:** 
  * Lakes → big raw data storage; 
  * Warehouse → analytics & reporting

Data Lake-> Result after Loading of ELT
- Data Lake = 集中存储各种原始数据，不要求立即清洗、转换或结构化
- 目的是保留 原始、完整、可追溯的数据，以备后续分析或 ETL/ELT 使用
- 数据格式可以是：CSV、JSON、Parquet、日志、图片、音视频等

Data Warehouse -> Result after Extraction of ETL 

---

###### **Batch Processing vs Streaming**

* **Batch:** Process a large size of data at once in chunks
  * Process Mode: Collect Data - Compute in Buiks - Output 
  * Characteristic
    * High Latency ( minutes to hours or even days)
    * High fault tolerance, suitable for large-scale computation 
    * Can recompute the batch, suitable for complex aggregations 
  * Use Case: Daily log analysis, reporting, ETL 
* **Streaming:** Process live data continuously as it arrives as small micro-batches 
  * Process Mode: Data arrives -> process immediately -> output 
  * Characteristic
    * Low Latency(seconds)
    * Real-time, Continuous Processing 
    * Requirement to high throughput and state management 
  * Use Case: Real-time clickstream analysis, monitoring, live logging 

* **Real-world use:** Batch → daily report, Streaming → live carpool tracking

| 特性   | Batch Processing          | Streaming Processing                  |
| ---- | ------------------------- | ------------------------------------- |
| 数据输入 | 大块历史数据                    | 实时流式数据                                |
| 延迟   | 高（分钟-小时）                  | 低（毫秒-秒）                               |
| 容错   | 重算批次即可                    | 状态管理复杂，需 checkpoint                   |
| 使用场景 | 报表、历史分析、ETL               | 实时监控、推荐、告警                            |
| 计算粒度 | 批次                        | 单条或微批                                 |
| 技术栈  | Hadoop, Spark Batch, Hive | Spark Streaming, Flink, Kafka Streams |


---

###### **Star Schema vs Snowflake Schema**

* **Star Schema:** Fact table at center, denormalized dimension tables

  * Fast for queries, simpler BI reports

* **Snowflake Schema:** Dimension tables normalized

  * Saves space, but slower joins

* **BI preference:** Star schema → easier aggregations and faster query performance

| Schema               | Description                                                                                            |
| -------------------- | ------------------------------------------------------------------------------------------------------ |
| **Star Schema**      | A central fact table connected directly to denormalized dimension tables. Simple, flat design.         |
| **Snowflake Schema** | A fact table connected to normalized dimension tables, which may be further split into sub-dimensions. |

| Feature             | Star Schema                      | Snowflake Schema                               |
| ------------------- | -------------------------------- | ---------------------------------------------- |
| Dimension Tables    | Denormalized                     | Normalized (may have sub-dimensions)           |
| Complexity          | Simple, easy to understand       | More complex, hierarchical                     |
| Query Performance   | Faster (fewer joins)             | Slower (more joins required)                   |
| Storage Requirement | Higher (data redundancy)         | Lower (less redundancy)                        |
| Maintenance         | Easier                           | Harder (updates in multiple tables)            |
| Example Use Case    | OLAP dashboards, quick reporting | Data warehouses requiring strict normalization |

Star Schema 
```
       维度表1
          │
维度表2 ── 事实表 ── 维度表3
          │
       维度表4
```

Snowflake Schema 
```
         子维表1
            │
         维度表1
            │
事实表 ── 维度表2 ── 子维表2
```
雪花模式将维度表规范化拆分，星型模式维度表扁平化存储。



In [ ]:
# 1. percentage of rides when it's carpool
select 100*sum(case when ride_type == 'carpool' then 1 else 0)/count(*) from fct_ride;

# 2. percentage of vehicle whose rides more carpool
# 1 vehicle : n rides 
with rides_total as (
    select vehicle_id,
            sum(case when ride_type == 'carpool' then 1 else 0) as carpool,
            sum(case when ride_type == 'regular' then 1 else 0) as regular
    from fct_ride
    group by vehicle_id 
)
select 100*(sum(case when carpool>regular then 1 else 0))/count(*) from rides_total


# 3. highest vehicle usage time 
select vehicle_id, sum(TIMESTAMP(SECOND, start_time, end_time)) as totals
from fct_ride
group by vehicle_id 
order by totals desc 
limit 1 